In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt

import os
for dirname, _, filenames in os.walk('/kaggle/working'): 
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/working/__notebook__.ipynb
/kaggle/input/ubiquant-market-prediction/example_sample_submission.csv
/kaggle/input/ubiquant-market-prediction/example_test.csv
/kaggle/input/ubiquant-market-prediction/train.csv
/kaggle/input/ubiquant-market-prediction/ubiquant/competition.cpython-37m-x86_64-linux-gnu.so
/kaggle/input/ubiquant-market-prediction/ubiquant/__init__.py


In [2]:
filename = '/kaggle/input/ubiquant-market-prediction/train.csv'

In [3]:
met=[]
dat=[]

dtypes = {'row_id':'category'
          , 'time_id':'int16'
          , 'investment_id':'int16'
          , 'target': 'float16'}
dtypes.update(dict.fromkeys([f'f_{i}' for i in range(300)],'float16'))

for chunk in pd.read_csv(filename, chunksize=100000,na_values='nan',dtype = dtypes):
    # load the metadata seperate from the features (ie variables)
    #met.append(chunk.iloc[:,0:4])
    dat.append(chunk)
    #break  # break  here load less data and run some EDA - to be removed

df = pd.concat(dat)

met = df.iloc[:,0:4]  # other data than the predictive features (incl. the target variable)
df = df.drop(['row_id','investment_id','target'], axis = 1)  #remove some variables but keep time_id for now

print('--- METADATA ---')
print(met.shape, met.info(memory_usage='deep'))
print(met.dtypes)

print('\n--- Features dataset ---')
print(df.shape, df.info(memory_usage='deep'))
print(df.dtypes)

--- METADATA ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3141410 entries, 0 to 3141409
Data columns (total 4 columns):
 #   Column         Dtype  
---  ------         -----  
 0   row_id         object 
 1   time_id        int16  
 2   investment_id  int16  
 3   target         float16
dtypes: float16(1), int16(2), object(1)
memory usage: 212.3 MB
(3141410, 4) None
row_id            object
time_id            int16
investment_id      int16
target           float16
dtype: object

--- Features dataset ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3141410 entries, 0 to 3141409
Columns: 301 entries, time_id to f_299
dtypes: float16(300), int16(1)
memory usage: 1.8 GB
(3141410, 301) None
time_id      int16
f_0        float16
f_1        float16
f_2        float16
f_3        float16
            ...   
f_295      float16
f_296      float16
f_297      float16
f_298      float16
f_299      float16
Length: 301, dtype: object


In [4]:
# the "groupby" craches the run So I went through a first step to select randomly datapoint within the dataset
reduce_index = df.groupby('time_id').apply(lambda x: x.sample(5)).index  #.reset_index(drop=True)   # get the indeces here because I need to use it on target value too
dfred = df.iloc[reduce_index.get_level_values(1)]
print(dfred.shape)
dfred.head()
del df

(6055, 301)


In [5]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X = np.array(dfred.iloc[:,1:])
Y = np.array(met.target.iloc[reduce_index.get_level_values(1)])

scaler.fit(X)
X = scaler.transform(X)

print(np.shape(X), np.shape(Y))

del dfred
 
#??? do we need to know if it is needed or just goog practice
#!!! Problem here is that we will run out of memory by creating new arrays

(6055, 300) (6055,)


In [6]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

del X
del Y

(4238, 300) (1817, 300) (4238,) (1817,)


In [7]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()
rf.fit(x_train, y_train)   #fit the model
rf_confidence = rf.score(x_test, y_test)  #test against the test dataset and give the coefficient of determination of the prediction (best score is 1 & score can be <0)
print('confidence: ', rf_confidence)  #or accuracy

confidence:  -0.03084117156967725


In [8]:
import ubiquant
env = ubiquant.make_env()   # initialize the environment
iter_test = env.iter_test()    # an iterator which loops over the test set and sample submission
for (test_df, sample_prediction_df) in iter_test:
    test_df.drop(['row_id','investment_id'], axis=1, inplace=True)
    #print(test_df.shape)
    sample_prediction_df['target'] = rf.predict(test_df)  # make your predictions here
    env.predict(sample_prediction_df)   # register your predictions

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.
